In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis, variation

In [2]:
base_directory = "../results/image_classification/"
results_column = "test_accuracy"
round_digits = 4
cvar_alpha = 0.05

In [3]:
experiments_by_model_and_dataset = [
    # EX1 - Small Image Classification
    [
        [
            "ResNet20-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ResNet56-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ResNet110-cifar10-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTS8-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ViTB8-cifar10-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ResNet20-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ResNet56-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ResNet110-cifar100-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTS8-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ViTB8-cifar100-idun-A100-PyTorch-ngc2312.csv",
        ],
    ],
    # EX2 - Large Image Classification
    [
        [
            "ResNet18-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "ResNet50-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "ResNet101-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTTiny16-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "ViTS16-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "ViTB16-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTTiny16-oxford_flowers102-idun-A100-PyTorch-ngc2312-pretrained.csv",
            "ViTS16-oxford_flowers102-idun-A100-PyTorch-ngc2312-pretrained.csv",
            "ViTB16-oxford_flowers102-idun-A100-PyTorch-ngc2312-pretrained.csv",
        ],

        [
            "ResNet18-uc_merced-idun-A100-PyTorch-ngc2312.csv",
            "ResNet50-uc_merced-idun-A100-PyTorch-ngc2312.csv",
            "ResNet101-uc_merced-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTTiny16-uc_merced-idun-A100-PyTorch-ngc2312.csv",
            "ViTS16-uc_merced-idun-A100-PyTorch-ngc2312.csv",
            "ViTB16-uc_merced-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTTiny16-uc_merced-idun-A100-PyTorch-ngc2312-pretrained.csv",
            "ViTS16-uc_merced-idun-A100-PyTorch-ngc2312-pretrained.csv",
            "ViTB16-uc_merced-idun-A100-PyTorch-ngc2312-pretrained.csv",
        ],
    ],
    # EX3 - Learning Rate Warmup Comparison
    [
        [
            "ResNet20-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "learning_rate/ResNet20LR-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ResNet56-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "learning_rate/ResNet56LR-cifar10-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ResNet20-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "learning_rate/ResNet20LR-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ResNet56-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "learning_rate/ResNet56LR-cifar100-idun-A100-PyTorch-ngc2312.csv",
        ],
    ],
    # EX4 - Random train/val/test splits
    [
        [
            "ResNet20-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_01-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_02-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_03-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_04-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_05-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_06-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_07-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_08-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet20_09-cifar10-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ResNet50-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_01-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_02-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_03-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_04-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_05-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_06-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_07-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_08-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "dataset_splits/ResNet50_09-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
        ],
    ],
    # EX5 - PyTorch vs TensorFlow
    [
        [
            "ResNet20-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ResNet56-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ResNet110-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ResNet20-cifar10-idun-A100-TensorFlow-ngc2312.csv",
            "ResNet56-cifar10-idun-A100-TensorFlow-ngc2312.csv",
            "ResNet110-cifar10-idun-A100-TensorFlow-ngc2312.csv",
        ],
        [
            "ResNet20-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ResNet56-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ResNet110-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ResNet20-cifar100-idun-A100-TensorFlow-ngc2312.csv",
            "ResNet56-cifar100-idun-A100-TensorFlow-ngc2312.csv",
            "ResNet110-cifar100-idun-A100-TensorFlow-ngc2312.csv",
        ],
    ],
    # EX6 - Increasing Epochs
    [
        [
            "epochs/ResNet20_e010-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e020-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e030-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e040-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e050-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e075-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e100-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e125-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e150-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e175-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ResNet20_e200-cifar10-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "epochs/ViTS8_e010-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e020-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e030-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e040-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e050-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e075-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e100-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e125-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e150-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e175-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "epochs/ViTS8_e200-cifar10-idun-A100-PyTorch-ngc2312.csv",
        ],
    ],
]

In [4]:
def calculate_cvar(dataset, alpha):
    # alpha = 0.1 = 90% 
    # alpha = 0.05 = 95%
    # alpha = 0.01 = 99%    

    dataset.sort()
    var = np.quantile(dataset, alpha)
    cvar = dataset[dataset <= var].mean().round(round_digits)
    return(cvar)

In [5]:
def save_kde(data):

    max_y = 0  # Initialize a variable to track the maximum y-axis limit across all plots

    # Set the number of columns and rows for the subplots
    ncols = 2
    nrows = math.ceil(len(data) / ncols)  # Calculate the number of rows needed based on the data size

    # Dynamically adjust figure size based on the number of rows
    figsize = (20, 10 * nrows)

    # Create subplots with the determined number of rows and columns
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    fig.tight_layout(pad=7)  # Add padding between subplots

    # Ensure `ax` is always treated as a 2D array even if there's only one row or column
    ax = np.atleast_2d(ax)

    x_mid = np.zeros((nrows, ncols))  # To store the midpoints of x-axis for each subplot
    x_range = 0  # Initialize variable to track the largest x-axis range

    # Loop through the data and create a KDE plot for each item
    for idx, title in enumerate(data):
        column = idx % ncols  # Determine the column index for the subplot
        row = idx // ncols    # Determine the row index for the subplot

        # Plot KDEs for each model in the inner dictionary of `data[title]`
        for model, values in data[title].items():
            sns.kdeplot(values, ax=ax[row, column], fill=True, label=model)

        ax[row, column].legend(fontsize=25)  # Add a legend with larger font size
        ax[row, column].set_title(title, fontsize=25)  # Set the title for the subplot
        ax[row, column].set_xlabel("Top-1 Accuracy", fontsize=25)  # Label the x-axis
        ax[row, column].set_ylabel("Density", fontsize=25)  # Label the y-axis
        ax[row, column].tick_params(labelsize=20)  # Adjust tick label sizes

        # Get current x-axis limits and calculate the midpoint
        x_lim = ax[row, column].get_xlim()
        x_mid[row, column] = (x_lim[0] + x_lim[1]) / 2  # Calculate midpoint of the x-axis range

        # Update the largest x-axis range found across all plots
        if (x_lim[1] - x_lim[0]) > x_range:
            x_range = x_lim[1] - x_lim[0]

        # Get current y-axis limits and update the global maximum y-axis limit
        y_lim = ax[row, column].get_ylim()
        if y_lim[1] > max_y:
            max_y = y_lim[1]

    # Set consistent x and y limits for all subplots
    for row_idx in range(nrows):
        for col_idx in range(ncols):
            # Check if the current subplot corresponds to actual data
            if row_idx * ncols + col_idx < len(data):
                # Set x-axis limits for each plot, ensuring the range stays within [0, 1]
                x_min = x_mid[row_idx, col_idx] - x_range / 2
                x_max = x_mid[row_idx, col_idx] + x_range / 2
                if x_min < 0:  # Adjust x_min and x_max if necessary to stay within bounds
                    x_min = 0
                    x_max = x_range
                if x_max > 1:
                    x_min = 1 - x_range
                    x_max = 1

                ax[row_idx, col_idx].set_xlim([x_min, x_max])

                # Set the y-axis limit for each plot to ensure consistency across all plots
                ax[row_idx, col_idx].set_ylim([0, max_y])

    plt.show()  # Display the plots

    # Save the figure to a file, with the file name based on the first three letters of the title
    fig.savefig(title[0:3].lower() + "_kde.png", pad_inches=0.1, bbox_inches='tight')

In [6]:
def save_boxplot(data):
    # Set the number of columns and rows for the subplots
    ncols = 1
    nrows = math.ceil(len(data) / ncols)  # Calculate the number of rows needed based on the data size

    # Dynamically adjust figure size based on the number of rows
    figsize = (20, 5 * nrows)

    # Create subplots with the determined number of rows and columns
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    fig.tight_layout(pad=7)  # Add padding between subplots

    # If there's only one subplot, `ax` is not an array, so convert it to a list for consistent handling
    if nrows == 1 and ncols == 1:
        ax = [ax]
    else:
        ax = np.ravel(ax)  # Flatten the `ax` array for easier iteration

    # Loop through the data and create a box plot for each item
    for idx, title in enumerate(data):
        sns.boxplot(pd.DataFrame.from_dict(data[title]), ax=ax[idx])
        ax[idx].set_title(title, fontsize=16)               # Set the title for each subplot
        ax[idx].set_ylabel("Top-1 Accuracy", fontsize=16)   # Label the y-axis
        ax[idx].tick_params(labelsize=14)                   # Adjust tick label size

        # Hide top and right spines for better aesthetics
        ax[idx].spines["top"].set_visible(False)
        ax[idx].spines["right"].set_visible(False)

    plt.show()

    fig.savefig(title[0:3].lower() + "_box.png", pad_inches=0.1, bbox_inches='tight')

In [7]:
def save_cvar(data):
    # Set the number of columns and rows for the subplots
    # 2 columns and as many rows as needed to fit all the data
    ncols = 2
    nrows = 0

    # Calculate the number of rows needed based on the data size
    # Iterate over each title in the data and sum up the number of rows needed
    for idx, title in enumerate(data):
        runs = len(data[title])  # Number of runs associated with the current title
        nrows += math.ceil(runs / ncols)  # Calculate the number of rows for the current title

    # Initialize variables to track the maximum y-axis limit and x-axis range across all plots
    max_y = 0  # Maximum y-axis value
    x_mid = np.zeros((nrows, ncols))  # To store the midpoints of the x-axis for each subplot
    x_range = 0  # Largest x-axis range across all plots

    # Set figure size based on the number of rows
    figsize = (20, 5 * nrows)  # Adjust width and height of the figure dynamically
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)  # Create subplots
    fig.tight_layout(pad=7)  # Adjust padding between plots

    # Uncomment the following line to add a global title (if necessary)
    #fig.suptitle(title, fontsize=25)

    plt.legend(fontsize='x-large', title_fontsize='40')  # Set legend font sizes

    row_run_offset = 0  # To manage row indices across different runs

    # Loop through each title in the data and generate plots
    for idx, title in enumerate(data):
        
        # Iterate through each run for the current title
        for idy, run_name in enumerate(data[title]):
            # Calculate the subplot's row and column indices
            column = idy % ncols  # Column index (modulus ensures alternating between columns)
            row = (idy // ncols) + row_run_offset  # Row index (adjusted by the offset for each title)

            # Calculate the mean and CVaR (Conditional Value at Risk) for the current run
            dataset_mean = data[title][run_name].mean()
            cvar = calculate_cvar(data[title][run_name], cvar_alpha)  # Custom function to calculate CVaR

            # Create a Kernel Density Estimate (KDE) plot for the run's data
            sns.kdeplot(data[title][run_name], 
                        ax=ax[row, column], 
                        fill=True, 
            )

            # Add vertical lines for the mean and CVaR
            ax[row, column].axvline(dataset_mean, color='red', linestyle='solid', 
                                    label="Mean: %.2f%% " % (dataset_mean * 100))  # Red solid line for the mean
            ax[row, column].axvline(cvar, color='red', linestyle='dashed', 
                                    label="CVaR: %.2f%% " % (cvar * 100))  # Red dashed line for CVaR

            # Add legend to the current plot
            ax[row, column].legend(fontsize=25)

            # Set plot title and axis labels
            ax[row, column].set_title(run_name, fontsize=25)  # Set the run name as the plot title
            ax[row, column].set_xlabel("Top-1 Accuracy", fontsize=25)  # Label for x-axis
            ax[row, column].set_ylabel("Density", fontsize=25)  # Label for y-axis
            
            ax[row, column].tick_params(labelsize=20)  # Set tick size for better readability

            # Get current x-axis limits and calculate the midpoint
            x_lim = ax[row, column].get_xlim()
            x_mid[row, column] = (x_lim[0] + x_lim[1]) / 2  # Calculate and store the midpoint of the x-axis range

            # Update the largest x-axis range found across all plots
            if (x_lim[1] - x_lim[0]) > x_range:
                x_range = x_lim[1] - x_lim[0]

            # Get current y-axis limits and update the global maximum y-axis limit
            y_lim = ax[row, column].get_ylim()
            if y_lim[1] > max_y:
                max_y = y_lim[1]

        # After processing all runs for the current title, update row_run_offset to start on a new row
        row_run_offset = row + 1

    # Ensure consistent x and y limits across all subplots
    for row_idx in range(nrows):
        for col_idx in range(ncols):

            # Hide empty graphs (subplots that have no data)
            if ax[row_idx, col_idx].get_xlim() == (0.0, 1.0):
                ax[row_idx, col_idx].axis('off')  # Turn off axis for empty plots
                continue

            # Set x-axis limits to be consistent across all subplots
            x_min = x_mid[row_idx, col_idx] - x_range / 2
            x_max = x_mid[row_idx, col_idx] + x_range / 2
            if x_min < 0:  # Ensure x_min is within bounds
                x_min = 0
                x_max = x_range
            if x_max > 1:  # Ensure x_max is within bounds
                x_min = 1 - x_range
                x_max = 1

            # Apply consistent x-axis limits
            ax[row_idx, col_idx].set_xlim([x_min, x_max])

            # Set consistent y-axis limits across all subplots
            ax[row_idx, col_idx].set_ylim([0, max_y])

    # Display the figure
    plt.show()

    # Save the figure with the title as part of the filename
    fig.savefig(title[0:3].lower() + "_cvar.png", pad_inches=0.1, bbox_inches='tight')


In [ ]:
# Save the summary statistics so they can be saved to a CSV file
summary_statistics = []

for idx, experiment_list in enumerate(experiments_by_model_and_dataset):

    experiment_data = {}

    experiment_number = "EX" + str(idx + 1) + ":"
    
    for experiments in experiment_list:
        # Start the title with the experiment number
        title = experiment_number

        # Include model in title
        if "resnet" in experiments[0].lower():
            title += " ResNet"
        elif "vit" in experiments[0].lower():
            title += " ViT"

        # Save the individual results for the plots
        results_values = {}

        # Loop through the individual experiments
        for experiment in experiments:
            df = pd.read_csv(base_directory + experiment)

            # Get the 100 results
            results = df[results_column].values
            # Calculate the summary statistics
            dist_mean = np.mean(results).round(round_digits)
            dist_median = np.median(results).round(round_digits)
            dist_min = np.min(results).round(round_digits)
            dist_max = np.max(results).round(round_digits)
            dist_range = (dist_max - dist_min).round(round_digits)
            q_1 = np.quantile(results, 0.25).round(round_digits)
            q_3 = np.quantile(results, 0.75).round(round_digits)
            dist_std = np.std(results).round(round_digits)
            c_v = variation(results).round(round_digits)
            cvar = calculate_cvar(results, cvar_alpha)
            skewness = skew(results).round(round_digits)
            k_value = kurtosis(results).round(round_digits)

            # Get the model and dataset from the experiment name
            dataset = experiment.split("-")[1]
            model = experiment.split("-")[0]

            # Add the dataset to the title if it is not already there
            if dataset not in title:
                title = title + " " + dataset

            # Add whether the model is pretrained or not for the ViT 16 models
            if "vit" in model.lower() and "16" in model.lower():
                if "pretrained" in experiment:
                    model += " (Pretrained)"
                    if "pretrained" not in title.lower():
                        title += " (Pretrained)"
                else:
                    model += " (Random)"
                    if "random" not in title.lower():
                        title += " (Random)"

            # Remove the subdirectory name for the EX3 models
            if "ex3" in title.lower():
                model = model.replace("learning_rate/", "")

            # Remove the subdirectory name for the EX4 models        
            if "ex4" in title.lower():
                model = model.replace("dataset_splits/", "")

            # Add whether the model is TensorFlow or PyTorch for the EX5 experiments
            if "ex5" in title.lower():
                if "tensorflow" in experiment.lower():
                    model += " (TensorFlow)"
                else:
                    model += " (PyTorch)"

            # Remove the subdirectory name for the EX6 models        
            if "ex6" in title.lower():
                model = model.replace("epochs/", "")

            # Save the results for the plot
            results_values[model] = results

            # Save the summary statistics
            summary_statistics.append([dataset, model, dist_mean, dist_median, dist_min, dist_max, dist_range, q_1, q_3, dist_std, c_v, cvar, skewness, k_value])

        # Save the results for the experiment
        experiment_data[title] = results_values

    # Plot the results for the experiment as a KDE histogram
    save_kde(experiment_data)
    save_boxplot(experiment_data)
    save_cvar(experiment_data)


In [10]:
summary_statistics_df = pd.DataFrame(summary_statistics, columns=["Dataset", "Model", "Mean", "Median", "Min", "Max", "Range", "25th_percentile", "75th_percentile", "Std", "Coefficient of Variation", "CVaR 95%", "Skewness", "Kertosis"])
summary_statistics_df.to_csv("image_summary_statistics.csv", index=False)